In [2]:
# pip install sentence-transformers

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
from sentence_transformers import SentenceTransformer

bert_base = 'bert-base-nli-mean-tokens'
all_mini = 'all-MiniLM-L6-v2'
sentence_t5 = 'Sentence-T5'

model = SentenceTransformer(all_mini)
# sentence_embeddings = model.encode('This is an example')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [4]:
titles = pd.read_csv('../assets/titles_200p.csv').drop_duplicates()
tags = pd.read_csv('../assets/tags_200p.csv')

In [53]:
# titles = titles.merge(tags.groupby('title_id').agg({'tag_name':list}).reset_index(), how = 'left', on = 'title_id')
# titles['synopsis'] = titles['synopsis'] + titles['tag_name'].astype(str)

In [54]:
titles.head(2)

,title_id,title_english,title_romaji,type,duration,start_year,chapters,volume,publishing_status,country,...,count_CURRENT,count_PLANNING,count_COMPLETED,count_DROPPED,count_PAUSED,ranking_RATED,ranking_POPULAR,synopsis,cover_image_url,tag_name
0,30002,Berserk,Berserk,MANGA,NaN,1989.0,NaN,NaN,RELEASING,JP,...,53949,35638,14376,1686,10616,1.0,6.0,"His name is Guts, the Black Swordsman, a feare...",{'large': 'https://s4.anilist.co/file/anilistc...,"[Tragedy, Seinen, Revenge, Male Protagonist, D..."
1,31706,NaN,JoJo no Kimyou na Bouken: Steel Ball Run,MANGA,NaN,2004.0,95.0,24.0,FINISHED,JP,...,6245,14526,30266,352,1466,1.0,1.0,"Originally presented as an unrelated story, th...",{'large': 'https://s4.anilist.co/file/anilistc...,"[Super Power, Male Protagonist, Seinen, Conspi..."


In [5]:
titles.shape

(9011, 35)

In [56]:
titles.iloc[0].synopsis

'His name is Guts, the Black Swordsman, a feared warrior spoken of only in whispers. Bearer of a gigantic sword, an iron hand, and the scars of countless battles and tortures, his flesh is also indelibly marked with The Brand, an unholy symbol that draws the forces of darkness to him and dooms him as their sacrifice. But Guts won\'t take his fate lying down; he\'ll cut a crimson swath of carnage through the ranks of the damned—and anyone else foolish enough to oppose him! Accompanied by Puck the Elf, more an annoyance than a companion, Guts relentlessly follows a dark, bloodstained path that leads only to death...or vengeance.<br>\n<br>\n(Source: Dark Horse)<br>\n<br>\n<i>Notes:<br>\n- Volumes 1-5 contain the 16 prequel chapters 0A - 0P.<br>\n- <a href="https://anilist.co/manga/95821/">Chapter 83</a> was omitted from Volume 13 due to the author’s request.<br>\n- Volume 14 includes “<a href="https://anilist.co/manga/36408/">Berserk: The Prototype</a>”.<br>\n- Due to the author\'s passin

In [10]:
titles.synopsis.fillna('').reset_index(drop= True)

0       His name is Guts, the Black Swordsman, a feare...
1       Originally presented as an unrelated story, th...
2       <i>Gintama: THE FINAL</i> is the 3rd and final...
3       As a child, Monkey D. Luffy was inspired to be...
4       After last season’s revelations, the Soma fami...
                              ...                        
9006    Out of the blue, childhood friend Takaoka conf...
9007    Official music video for IDOLiSH7's song WiSH ...
9008    Kyousuke Kousaka, a normal seventeen-year-old ...
9009                                                     
9010    Makia is a noble lady descended from an infamo...
Name: synopsis, Length: 9011, dtype: object

In [5]:
popular_title = titles.loc[lambda x : x.popularity > 10000]['title_id'].tolist()

In [6]:
sentence_embeddings = model.encode(titles.synopsis.fillna('').reset_index(drop= True))

In [7]:
sentence_embeddings.shape

(9011, 384)

In [8]:
wv = model.encode(['genius protagonists who are undefeatable'])
rec_df = pd.Series(cosine_similarity(wv, sentence_embeddings).flatten(), index = titles.title_id, name = 'similarity').sort_values(ascending = False).loc[lambda x : x.index.isin(popular_title)].to_frame()
rec_df.reset_index().merge(titles[['title_id','title_english','title_romaji','synopsis']], how = 'left', on = 'title_id').head(20)

,title_id,similarity,title_english,title_romaji,synopsis
0,117343,0.501518,Talentless Nana,Munou na Nana,It is the year 20XX. Earth has been assaulted ...
1,124786,0.501246,Godzilla Singular Point,Godzilla: Singular Point,"This series features an original story, which ..."
2,21459,0.458755,My Hero Academia,Boku no Hero Academia,What would the world be like if 80 percent of ...
3,534,0.447221,The Slayers,Slayers,Welcome to a world where magic reigns and mons...
4,108553,0.437743,My Hero Academia: Heroes Rising,Boku no Hero Academia THE MOVIE: Heroes:Rising,The second installment in the <i>Boku no Hero ...
5,3228,0.431912,NaN,Zoku Sayonara Zetsubou Sensei,"More crazy antics abound as Nozomu Itoshiki, a..."
6,21127,0.429644,NaN,Steins;Gate 0,"The eccentric, self-proclaimed mad scientist R..."
7,110229,0.423615,We Never Learn!: BOKUBEN Season 2,Bokutachi wa Benkyou ga Dekinai!,"To get a scholarship, Nariyuki becomes a reluc..."
8,10161,0.422728,No.6,NO.6,"In a near future world, after the last great w..."
9,101249,0.421453,Penguin Highway,Penguin Highway,Budding genius Aoyama is only in the 4th grade...


In [11]:
wv = model.encode(['dark mideval era vibes with witchcraft and magic'])
rec_df = pd.Series(cosine_similarity(wv, sentence_embeddings).flatten(), index = titles.title_id, name = 'similarity').sort_values(ascending = False).loc[lambda x : x.index.isin(popular_title)].to_frame()
rec_df.reset_index().merge(titles[['title_id','title_english','title_romaji','synopsis']], how = 'left', on = 'title_id').head(20)

,title_id,similarity,title_english,title_romaji,synopsis
0,33,0.473409,Berserk,Kenpuu Denki Berserk,Set during a time that very much resembles Eur...
1,20458,0.461811,The Irregular at Magic High School,Mahouka Koukou no Rettousei,Magic is a product of neither legends nor fair...
2,101024,0.459085,RADIANT,Radiant,"The story follows Seth, a boy who seeks to bec..."
3,100207,0.450823,The Great Jahy Will Not Be Defeated!,Jahy-sama wa Kujikenai!,"The Great Jahy, the Dark Realm’s second-in-com..."
4,132456,0.450823,The Great Jahy Will Not Be Defeated!,Jahy-sama wa Kujikenai!,"The Great Jahy, the Dark Realm’s second-in-com..."
5,97682,0.450168,Grimoire of Zero,Zero kara Hajimeru Mahou no Sho,"The ""grimoire fantasy"" series follows a witch ..."
6,4654,0.447287,A Certain Magical Index,Toaru Majutsu no Index,Set in a city of scientifically advanced super...
7,207,0.432809,Record of Lodoss War,Lodoss-tou Senki,Created from the aftermath of the last great b...
8,534,0.430394,The Slayers,Slayers,Welcome to a world where magic reigns and mons...
9,8086,0.428723,The Legend of the Legendary Heroes,Densetsu no Yuusha no Densetsu,"""Alpha Stigma"" are known to be eyes that can a..."


In [10]:
wv = model.encode(['heros saving lives and fighting crimes'])
rec_df = pd.Series(cosine_similarity(wv, sentence_embeddings).flatten(), index = titles.title_id, name = 'similarity').sort_values(ascending = False).loc[lambda x : x.index.isin(popular_title)].to_frame()
rec_df.reset_index().merge(titles[['title_id','title_english','title_romaji','synopsis']], how = 'left', on = 'title_id').head(20)

,title_id,similarity,title_english,title_romaji,synopsis
0,97842,0.604768,My Hero Academia: Vigilantes,Vigilante: Boku no Hero Academia ILLEGALS,"In the world of My Hero Academia, not everyone..."
1,21665,0.459276,B: The Beginning,B: The Beginning,"In a world powered by advanced technology, cri..."
2,100954,0.448476,Sweet Home,Sweet Home,"After an unexpected family tragedy, a reclusiv..."
3,1827,0.442868,Moribito: Guardian of the Spirit,Seirei no Moribito,"Balsa’s prowess in battle is legendary, but th..."
4,97888,0.439024,BAKI,Baki,"The protagonist, Baki Hanma, trains with an in..."
5,108307,0.432472,PSYCHO-PASS 3,PSYCHO-PASS 3,"""Justice"" opens up a new world. In the near fu..."
6,30731,0.427239,Eden: It's an Endless World!,Eden: It's an Endless World!,"In the near future, a large portion of humanit..."
7,21428,0.426394,Grimgar of Fantasy and Ash,Hai to Gensou no Grimgar,"Fear, survival, instinct. Thrown into a foreig..."
8,97616,0.425964,TO BE HERO,Tu Bian Yingxiong,"Handsome, divorced and with a teen daughter li..."
9,10418,0.425393,Deadman Wonderland: The Red Knife Wielder,Deadman Wonderland: Akai Knife Tsukai,Two years after the catastrophic tidal wave th...


In [12]:
wv = model.encode(['cops with undercover mission'])
rec_df = pd.Series(cosine_similarity(wv, sentence_embeddings).flatten(), index = titles.title_id, name = 'similarity').sort_values(ascending = False).loc[lambda x : x.index.isin(popular_title)].to_frame()
rec_df.reset_index().merge(titles[['title_id','title_english','title_romaji','synopsis']], how = 'left', on = 'title_id').head(20)

,title_id,similarity,title_english,title_romaji,synopsis
0,98505,0.443270,Princess Principal,Princess Principal,"The ""spy action"" series follows five girls in ..."
1,20773,0.430651,GANGSTA.,GANGSTA.,"In the city of Ergastulum, a shady ville fille..."
2,54903,0.415937,GANGSTA.,GANGSTA.,"In the city of Ergastulum, a shady ville fille..."
3,101595,0.410588,Princess Principal: ~Crown Handler~ 1,Princess Principal: Crown Handler - Chapter 1,The first out of six movie sequels to the TV s...
4,650,0.397410,Gunsmith Cats,Gunsmith Cats,"Rally Vincent knows her weapons well, while he..."
5,140960,0.390943,SPY x FAMILY,SPY×FAMILY,Everyone has a part of themselves they cannot ...
6,570,0.383700,Jin-Roh: The Wolf Brigade,JIN-ROH,In a very different Tokyo from the one we know...
7,21679,0.383291,Bungo Stray Dogs 2,Bungou Stray Dogs 2,"Despite their differences in position, three m..."
8,731,0.381093,Interstella 5555: The 5tory of the 5ecret 5tar...,Interstella 5555: THE 5TORY OF THE 5ECRET 5TAR...,"This film is a visual companion piece to ""Disc..."
9,99470,0.376066,NaN,Grisaia: Phantom Trigger THE ANIMATION,"Following the Heath Oslo incident, the existen..."


In [13]:
wv = model.encode(['highschool romance with super handsome senpai'])
rec_df = pd.Series(cosine_similarity(wv, sentence_embeddings).flatten(), index = titles.title_id, name = 'similarity').sort_values(ascending = False).loc[lambda x : x.index.isin(popular_title)].to_frame()
rec_df.reset_index().merge(titles[['title_id','title_english','title_romaji','synopsis']], how = 'left', on = 'title_id').head(20)

,title_id,similarity,title_english,title_romaji,synopsis
0,106434,0.559068,Hitomi-chan is Shy With Strangers,Hitomi-chan wa Hitomishiri,Tall and busty high school girl Takano Hitomi ...
1,120697,0.553704,"DON'T TOY WITH ME, MISS NAGATORO","Ijiranaide, Nagatoro-san","""A girl in a lower grade just made me cry!""<br..."
2,21686,0.523163,"Kiss Him, Not Me",Watashi ga Motete Dousunda,"Kae Serinuma is what you'd call a ""fujoshi."" W..."
3,99916,0.505634,Kase-san and Morning Glories,Asagao to Kase-san.,"A relationship is like a garden. It's hard, re..."
4,101992,0.502842,Sound! Euphonium: The Movie - Our Promise: A B...,Hibike! Euphonium: Chikai no Finale,The next piece begins! Kumiko is now in her se...
5,107987,0.501575,Whisper Me a Love Song,Sasayaku You ni Koi wo Utau,"Bubbly, energetic first-year high school stude..."
6,112981,0.494135,Kubo Won't Let Me Be Invisible,Kubo-san wa Mob wo Yurusanai,First year high schooler Junta Shiraishi is a ...
7,21701,0.491605,Scum's Wish,Kuzu no Honkai,Hanabi has loved her older friend Narumi for y...
8,137877,0.486456,GANBARE DOUKICHAN,"Ganbare, Douki-chan","Announced on Yom's Twitter, a web anime adapta..."
9,10460,0.484182,You and Me.,Kimi to Boku.,The story revolves around four teens &mdash; t...


In [16]:
wv = model.encode(['postwar psychological trauma and depression'])
rec_df = pd.Series(cosine_similarity(wv, sentence_embeddings).flatten(), index = titles.title_id, name = 'similarity').sort_values(ascending = False).loc[lambda x : x.index.isin(popular_title)].to_frame()
rec_df.reset_index().merge(titles[['title_id','title_english','title_romaji','synopsis']], how = 'left', on = 'title_id').head(20)

,title_id,similarity,title_english,title_romaji,synopsis
0,103047,0.382764,Violet Evergarden: the Movie,Violet Evergarden Movie,"While writing other people’s emotions, she may..."
1,578,0.339460,Grave of the Fireflies,Hotaru no Haka,"In the aftermath of a World War II bombing, tw..."
2,30003,0.326664,20th Century Boys,20th Century Boys,"Humanity, having faced extinction at the end o..."
3,2605,0.321420,NaN,Sayonara Zetsubou Sensei,Itoshiki Nozomu is always in despair! Even sim...
4,3783,0.320038,"the Garden of sinners Chapter 3: ever cry, nev...",Kara no Kyoukai: Tsuukaku Zanryuu,"July, 1998: One night, Mikiya helps a girl lyi..."
5,2593,0.307405,the Garden of sinners Chapter 1: Thanatos. (Ov...,Kara no Kyoukai: Fukan Fuukei,"September, 1998: In the city where Shiki lives..."
6,8142,0.306399,Colorful ~ The Motion Picture,Colorful,"""I"" died and was kicked out of the cycle of re..."
7,9989,0.305031,Anohana: The Flower We Saw That Day,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,Jinta Yadomi and his group of childhood friend...
8,15227,0.288769,In This Corner of the World,Kono Sekai no Katasumi ni,The story follows a young lady named Suzu Uran...
9,56736,0.281854,House of the Sun,Taiyou no Ie,"As a child, Mao spent all her time at Hiro’s h..."


In [18]:
wv = model.encode(['He is weak at first, but he is getting stronger'])
rec_df = pd.Series(cosine_similarity(wv, sentence_embeddings).flatten(), index = titles.title_id, name = 'similarity').sort_values(ascending = False).loc[lambda x : x.index.isin(popular_title)].to_frame()
rec_df.reset_index().merge(titles[['title_id','title_english','title_romaji','synopsis']], how = 'left', on = 'title_id').head(20)

,title_id,similarity,title_english,title_romaji,synopsis
0,263,0.482706,Fighting Spirit,Hajime no Ippo: THE FIGHTING!,Makunouchi Ippo has been bullied his entire li...
1,30007,0.469493,NaN,Hajime no Ippo,Makunouchi Ippo has been bullied his entire li...
2,130549,0.447181,RE-MAIN,RE-MAIN,Just after winning a national water polo title...
3,114960,0.440645,Mashle: Magic and Muscles,MASHLE,This is a world of magic where magic is used f...
4,264,0.382727,Fighting Spirit: Champion Road,Hajime no Ippo: Champion Road,"As the new Champion, Ippo must protect his tit..."
5,74347,0.368144,One-Punch Man,One Punch-Man,"In this new action-comedy, everything about a ..."
6,19647,0.364689,Fighting Spirit: Rising,Hajime no Ippo: Rising,Ippo continues to defend his Japanese Featherw...
7,30043,0.359943,Eyeshield 21,Eyeshield 21,Wimpy Sena Kobayakawa has been running away fr...
8,122807,0.359908,The Gymnastics Samurai,Taisou Zamurai,"Joutarou Aragaki, former member of the nationa..."
9,117193,0.355040,My Hero Academia Season 5,Boku no Hero Academia 5,The rivalry between Class 1-A and Class 1-B he...


In [24]:
pd.DataFrame(sentence_embeddings, index = titles.title_id).to_csv('feature_synopsis_minilm.csv')